In [2]:
import pandas as pd
import numpy as np
import os
import re
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from dateutil import parser

RAW_DATA_PATH = r"../data/raw"
PROCESSED_DATA_PATH = r"../data/processed"

In [3]:
plt.style.use("seaborn-v0_8-whitegrid")
sns.set_palette("Set2")
plt.rcParams["figure.figsize"] = (12, 8)
plt.rcParams["font.size"] = 12

pd.set_option("display.max_columns", None)

In [4]:
covid_data = pd.read_csv(os.path.join(RAW_DATA_PATH, "covid_19_data.csv"))

Kiểm tra lại bảng dữ liệu

In [5]:
covid_data.head(5)

,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
0,1,01/22/2020,Anhui,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
1,2,01/22/2020,Beijing,Mainland China,1/22/2020 17:00,14.0,0.0,0.0
2,3,01/22/2020,Chongqing,Mainland China,1/22/2020 17:00,6.0,0.0,0.0
3,4,01/22/2020,Fujian,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
4,5,01/22/2020,Gansu,Mainland China,1/22/2020 17:00,0.0,0.0,0.0


***Làm sạch cột ObservationDate và Last Update.***  

Vấn đề: Dữ liệu trong cột ObservationDate và Last Update là kiểu string, các giá trị không có sự thống nhất về định dạng để chuyển đổi sang datetime bằng phương pháp thủ công.  
Giải quyết: Dùng hàm parser.parse() từ thư viện dateutil.  
- Hàm parse() tự động chuyển đổi các giá trị có kiểu string về datetime nếu string đủ thông tin về thời gian.  
- Các giá trị không thể chuyển đổi sẽ gán dưới dạng pd.NaT (Not A Time) để kiểm tra.  

In [6]:
#Hàm chuyển đổi string về datetime
def safe_parse(date_str): 
    try:
        # Sử dụng parser của dateutil để chuyển đổi
        return parser.parse(date_str, dayfirst=True)  # dayfirst=True ưu tiên định dạng DD/MM/YYYY
    except (ValueError, TypeError):
        # Nếu không chuyển đổi được, trả về NaT (Not a Time)
        return pd.NaT

In [7]:
#Chuyển đổi 2 cột ObservationDate và Last Update về datetime
covid_data['ObservationDate'] = covid_data['ObservationDate'].apply(safe_parse)
covid_data['Last Update'] = covid_data['Last Update'].apply(safe_parse)

In [8]:
#Kiểm tra có giá trị nào không chuyển đổi thành công hay không
print(covid_data['ObservationDate'].isna().any())
print(covid_data['Last Update'].isna().any())

False
False


Kết quả: Không có dữ liệu lỗi, tất cả đã được chuyển thành công

#### Kiểm tra các giá trị null trong dữ liệu và fill các giá trị null

***Kiểm tra từng cột trong bảng có giá trị null hay không***

In [24]:
covid_data.isnull().sum()

SNo                    0
ObservationDate        0
Province/State     78103
Country/Region         0
Last Update            0
Confirmed              0
Deaths                 0
Recovered              0
dtype: int64

Ta thấy chỉ có cột Province/State có giá trị null

In [25]:
covid_data[covid_data["Province/State"].isnull()]

,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
35,36,2020-01-22,NaN,Japan,2020-01-22 17:00:00,2.0,0.0,0.0
36,37,2020-01-22,NaN,Thailand,2020-01-22 17:00:00,4.0,0.0,2.0
37,38,2020-01-22,NaN,South Korea,2020-01-22 17:00:00,1.0,0.0,0.0
39,40,2020-01-22,NaN,Kiribati,2020-01-22 17:00:00,0.0,0.0,0.0
75,76,2020-01-23,NaN,Japan,2020-01-23 17:00:00,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...
305831,305832,2021-05-29,NaN,Vietnam,2021-05-30 04:20:55,6908.0,47.0,2896.0
305832,305833,2021-05-29,NaN,West Bank and Gaza,2021-05-30 04:20:55,307838.0,3492.0,300524.0
305833,305834,2021-05-29,NaN,Yemen,2021-05-30 04:20:55,6731.0,1319.0,3399.0
305834,305835,2021-05-29,NaN,Zambia,2021-05-30 04:20:55,94751.0,1276.0,91594.0


Kiểm tra các giá trị của Province/State của một quốc gia ngẫu nhiên để tìm cách fill các giá trị null

In [28]:
covid_data[covid_data["Country/Region"] == "Japan"]["Province/State"].unique()

array([nan, 'Aichi', 'Akita', 'Aomori', 'Chiba', 'Ehime', 'Fukui',
       'Fukuoka', 'Fukushima', 'Gifu', 'Gunma', 'Hiroshima', 'Hokkaido',
       'Hyogo', 'Ibaraki', 'Ishikawa', 'Kagawa', 'Kagoshima', 'Kanagawa',
       'Kochi', 'Kumamoto', 'Kyoto', 'Mie', 'Miyagi', 'Miyazaki',
       'Nagano', 'Nagasaki', 'Nara', 'Niigata', 'Oita', 'Okayama',
       'Okinawa', 'Osaka', 'Port Quarantine', 'Saga', 'Saitama', 'Shiga',
       'Shimane', 'Shizuoka', 'Tochigi', 'Tokushima', 'Tokyo', 'Tottori',
       'Toyama', 'Unknown', 'Wakayama', 'Yamagata', 'Yamaguchi',
       'Yamanashi', 'Iwate'], dtype=object)

Ta thấy ở Nhật Bản có sử dụng Unknown có các giá trị bị mất\
=> Ta có thể sử dụng Unkwown cho các giá trị null

In [29]:
covid_data.fillna({"Province/State": "Unknown"}, inplace=True)

Kiểm tra lại dataframe

In [30]:
covid_data.isnull().sum()

SNo                0
ObservationDate    0
Province/State     0
Country/Region     0
Last Update        0
Confirmed          0
Deaths             0
Recovered          0
dtype: int64